In [43]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
print("TensorFlow version:", tf.__version__)
# Load the TensorBoard notebook extension
%load_ext tensorboard
import datetime
import shutil

TensorFlow version: 2.18.0
The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [44]:
 %reload_ext tensorboard
from tensorflow.keras import Model
from pathlib import Path
import pandas as pd

In [45]:
BATCH_SIZE = 64
BUFFER_SIZE = 128
LEARNING_RATE = 0.005
EPOCHS = 50

In [46]:
input_dir = Path('./data/prepared')
logs_path = Path('./data/logs')
if logs_path.exists():
  shutil.rmtree(logs_path)
logs_path.mkdir(parents=True)

X_train_name = input_dir / 'X_train.csv'
y_train_name = input_dir / 'y_train.csv'
X_test_name = input_dir / 'X_test.csv'
y_test_name = input_dir / 'y_test.csv'

X_train = pd.read_csv(X_train_name)
y_train = pd.read_csv(y_train_name)
X_test = pd.read_csv(X_test_name)
y_test = pd.read_csv(y_test_name)

train_ds = tf.data.Dataset.from_tensor_slices(
    (X_train, y_train)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(BATCH_SIZE)

In [47]:
X_train.shape

(3249, 8)

In [48]:
X_train

,year,fuel,seller_type,transmission,owner,brand,km_dr_gr,mpow_val
0,25,1,1,1,1,3,0,73
1,22,0,1,1,2,3,2,99
2,21,1,1,1,1,3,2,87
3,24,1,0,1,1,3,0,85
4,22,0,1,1,2,6,3,74
...,...,...,...,...,...,...,...,...
3244,25,1,1,1,1,3,0,84
3245,17,0,1,1,2,3,2,126
3246,16,0,1,1,2,3,3,102
3247,18,1,1,1,3,3,3,67


In [49]:
train_ds

<_BatchDataset element_spec=(TensorSpec(shape=(None, 8), dtype=tf.int64, name=None), TensorSpec(shape=(None, 1), dtype=tf.float64, name=None))>

In [50]:
@tf.keras.utils.register_keras_serializable()
class SomeModel(Model):
    def __init__(self, neurons_cnt=128, **kwargs):
        super(SomeModel, self).__init__(**kwargs)
        self.neurons_cnt = neurons_cnt  # Сохраняем значение параметра для конфигурации
        self.d_in = Dense(8, activation='relu')
        self.d1 = Dense(neurons_cnt, activation='relu')
        self.d2 = Dense(neurons_cnt, activation='relu')
        self.d_out = Dense(1, activation='sigmoid')

    def call(self, x):
        x = self.d_in(x)
        x = self.d1(x)
        x = self.d2(x)
        return self.d_out(x)

    def get_config(self):
        # Возвращаем параметры модели, включая кастомные
        config = super(SomeModel, self).get_config()
        config.update({
            "neurons_cnt": self.neurons_cnt  # Добавляем кастомный параметр в конфигурацию
        })
        return config

    @classmethod
    def from_config(cls, config):
        # Создаём экземпляр класса из конфигурации
        return cls(**config)

In [51]:
# Create an instance of the model
model = SomeModel(neurons_cnt=32)
model.build(input_shape=(None, 8))

D:\project\PycharmProjects\tpu-8e21-ai-basis\.venv\Lib\site-packages\keras\src\layers\layer.py:391: UserWarning: `build()` was called on layer 'some_model_2', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


In [52]:
loss_object = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.SGD(learning_rate=LEARNING_RATE)

In [53]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.MeanAbsoluteError(name='train_mae')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.MeanAbsoluteError(name='test_mae')

In [54]:
@tf.function
def train_step(input_vector, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(input_vector, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

In [55]:
@tf.function
def test_step(input_vector, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(input_vector, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [56]:
from tensorflow import keras

In [57]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = logs_path / 'gradient_tape' / current_time / 'train'
train_log_dir.mkdir(exist_ok=True, parents=True)
test_log_dir = logs_path / 'gradient_tape' / current_time / 'test'
test_log_dir.mkdir(exist_ok=True, parents=True)
train_summary_writer = tf.summary.create_file_writer(str(train_log_dir))
test_summary_writer = tf.summary.create_file_writer(str(test_log_dir))

logdir=logs_path / "fit" / datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
logdir.mkdir(exist_ok=True, parents=True)
fit_summary_writer = tf.summary.create_file_writer(str(logdir))

tf.summary.trace_on(graph=True, profiler=True, profiler_outdir=str(logdir))
for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  for (x_train, y_train) in train_ds:

    with fit_summary_writer.as_default():
      train_step(x_train, y_train)


  with train_summary_writer.as_default():
    tf.summary.scalar('loss', train_loss.result(), step=epoch)
    tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)

  for (x_test, y_test) in test_ds:
    test_step(x_test, y_test)

  with test_summary_writer.as_default():
    tf.summary.scalar('loss', test_loss.result(), step=epoch)
    tf.summary.scalar('mae', test_accuracy.result(), step=epoch)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test MAE: {}'
  print (template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result(),
                         test_loss.result(),
                         test_accuracy.result()))

  # Reset metrics every epoch
  train_loss.reset_state()
  test_loss.reset_state()
  train_accuracy.reset_state()
  test_accuracy.reset_state()

with fit_summary_writer.as_default():
  tf.summary.trace_export(
      name="my_func_trace",
      step=0,
      profiler_outdir=str(logdir)
  )

Epoch 1, Loss: 0.039132993668317795, Accuracy: 0.14939521253108978, Test Loss: 0.04088291898369789, Test MAE: 0.15056553483009338
Epoch 2, Loss: 0.03672472760081291, Accuracy: 0.14577481150627136, Test Loss: 0.04008947312831879, Test MAE: 0.14761309325695038
Epoch 3, Loss: 0.03477530553936958, Accuracy: 0.14181949198246002, Test Loss: 0.03518027067184448, Test MAE: 0.14558570086956024
Epoch 4, Loss: 0.03254890441894531, Accuracy: 0.13806921243667603, Test Loss: 0.03999989479780197, Test MAE: 0.14508046209812164
Epoch 5, Loss: 0.03201865404844284, Accuracy: 0.13643869757652283, Test Loss: 0.034850191324949265, Test MAE: 0.13822215795516968
Epoch 6, Loss: 0.029253942891955376, Accuracy: 0.13143622875213623, Test Loss: 0.033365003764629364, Test MAE: 0.13481563329696655
Epoch 7, Loss: 0.028787145391106606, Accuracy: 0.12954774498939514, Test Loss: 0.02881135232746601, Test MAE: 0.13047273457050323
Epoch 8, Loss: 0.027129817754030228, Accuracy: 0.12644565105438232, Test Loss: 0.02892227843

In [61]:
model.save('./data/models/mymodel.keras')

In [62]:
loaded_model = keras.models.load_model('./data/models/mymodel.keras', {'SomeModel': SomeModel})

In [63]:
loaded_model

<SomeModel name=some_model_2, built=True>

In [64]:
model

<SomeModel name=some_model_2, built=True>

In [65]:
# Let's check:
import numpy as np
np.testing.assert_allclose(
    model.predict(X_test),
    loaded_model.predict(X_test)
)
%tensorboard --logdir ./data/logs

59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 873us/step


Reusing TensorBoard on port 6006 (pid 21648), started 1:43:09 ago. (Use '!kill 21648' to kill it.)

In [67]:
%tensorboard --logdir ./data/logs/gradient_tape

Reusing TensorBoard on port 6006 (pid 37056), started 21:19:38 ago. (Use '!kill 37056' to kill it.)